In [1]:
import sys
sys.path.insert(0, '../')

In [2]:
from spacy.util import load_model

from src.config import PARAM, PATH_MODEL
from src.spacy_helpers import serialize_batch

batches = PARAM.lexisnexis.batches

In [3]:
nlp = load_model(PATH_MODEL)

In [4]:
for batch in batches:
    serialize_batch(nlp, batch)

leeuwarder_courant......: 100%|██████████| 303/303 [00:21<00:00, 14.64it/s]
